In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip -q gdrive/MyDrive/flower_data/models/model_mobilenetv2_regularized.zip
!unzip -q gdrive/MyDrive/flower_data/models/model_squeeze_excite_resnet.zip
!unzip -q gdrive/MyDrive/flower_data/models/model_pca_basd.zip
!unzip -q gdrive/MyDrive/flower_data/models/model_pca_sepconv.zip
!unzip -q gdrive/MyDrive/flower_data/models/model_lda_basd.zip
!unzip -q gdrive/MyDrive/flower_data/models/model_lda_sepconv.zip

replace model_mobilenetv2_regularized_oxford102/fingerprint.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace model_squeeze_excite_resnet/fingerprint.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace model_pca_sepconv/fingerprint.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace model_lda_based/fingerprint.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace model_lda_sepconv/fingerprint.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import pandas as pd
from matplotlib.patches import Patch
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from tensorflow import keras

In [ ]:
model_mn2 = keras.applications.MobileNetV2(
    input_shape=None, alpha=1.0, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)
layer_name = 'block_15_add'
feature_extractor = keras.models.Model(inputs=model_mn2.input, outputs=model_mn2.get_layer(layer_name).output)

14536120/14536120 [==============================] - 0s 0us/step


In [ ]:
feature_extractor.summary()

In [ ]:
model_mobilenetv2_regularized = tf.keras.models.load_model("model_mobilenetv2_regularized_oxford102")
model_squeeze_excite_resnet = tf.keras.models.load_model("model_squeeze_excite_resnet")
model_pca_based = tf.keras.models.load_model("model_pca_based")
model_pca_sepconv = tf.keras.models.load_model("model_pca_sepconv")
model_lda_based = tf.keras.models.load_model("model_lda_based")
model_lda_sepconv = tf.keras.models.load_model("model_lda_sepconv")

In [ ]:
model_mobilenetv2_regularized.summary()

In [ ]:
model_squeeze_excite_resnet.summary()

In [ ]:
model_pca_based.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 800)              3200      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               204800    
                                                                 
 elu (ELU)                   (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 480)               1

In [ ]:
model_pca_sepconv.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 5, 160)           640       
 ormalization)                                                   
                                                                 
 separable_conv1d (Separable  (None, 1, 320)           156320    
 Conv1D)                                                         
                                                                 
 elu (ELU)                   (None, 1, 320)            0         
                                                                 
 flatten (Flatten)           (None, 320)               0         
                                                                 
 dropout (Dropout)           (None, 320)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 320)              1

In [ ]:
model_lda_based.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 800)              3200      
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 256)               204800    
                                                                 
 elu_2 (ELU)                 (None, 256)               0         
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 480)              

In [ ]:
model_lda_sepconv.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_4 (Batc  (None, 5, 160)           640       
 hNormalization)                                                 
                                                                 
 separable_conv1d (Separable  (None, 1, 320)           156320    
 Conv1D)                                                         
                                                                 
 elu_4 (ELU)                 (None, 1, 320)            0         
                                                                 
 flatten (Flatten)           (None, 320)               0         
                                                                 
 dropout_4 (Dropout)         (None, 320)               0         
                                                                 
 batch_normalization_5 (Batc  (None, 320)             

In [ ]:
plants_names = []

for plant in os.listdir('gdrive/MyDrive/flower_data/encoded/train'):
    plants_names += [plant.split('.npy')[0]]

plants_names.sort()
plant_name_to_id = {plant: i for i, plant in enumerate(plants_names)}

In [ ]:
instances = {}
labels = {}

for set_kind in ["test", "train", "val"]:
    X_per_class = []
    y_per_class = []

    for i, file in enumerate(os.listdir(f"gdrive/MyDrive/flower_data/encoded/{set_kind}")):
        print(f"\r[{i}] {set_kind}/{file}", end='')
        plants_encoded = np.load(f"gdrive/MyDrive/flower_data/encoded/{set_kind}/{file}")
        plant_name = file.split('.npy')[0]

        X_per_class += [plants_encoded.reshape([plants_encoded.shape[0], 49, 160])]
        y_per_class += [plant_name_to_id[plant_name] for _ in range(plants_encoded.shape[0])]

    instances[set_kind] = np.concatenate(X_per_class)
    labels[set_kind] = np.array(y_per_class)

[101] val/16.npy

In [ ]:
n_classes = len(plants_names)

X_train = instances['train']
y_train = np.eye(n_classes)[labels['train']]

X_valid = instances['val']
y_valid = np.eye(n_classes)[labels['val']]

X_test = instances['test']
y_test = np.eye(n_classes)[labels['test']]

In [ ]:
model_mnv2 = keras.models.Sequential([
    keras.layers.Input(shape=(224, 224, 3), dtype=tf.uint8),
    keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float32)),
    keras.layers.Lambda(
        tf.keras.applications.mobilenet_v2.preprocess_input,
        name='preprocessing',
        input_shape=(224, 224, 3),
        dtype=tf.float32),
    feature_extractor,
    model_mobilenetv2_regularized
])

In [ ]:
model_seresnet = keras.models.Sequential([
    keras.layers.Input(shape=(224, 224, 3), dtype=tf.uint8),
    keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float32)),
    keras.layers.Lambda(
        tf.keras.applications.mobilenet_v2.preprocess_input,
        name='preprocessing',
        input_shape=(224, 224, 3),
        dtype=tf.float32),
    feature_extractor,
    model_squeeze_excite_resnet
])

In [ ]:
pca_per_filter = [
    PCA(n_components=5).fit(X_train[..., i])
    for i in range(X_train.shape[-1])
]

class PCATransformer(keras.layers.Layer):
    def __init__(self, means, components, **kwargs):
        super().__init__(**kwargs)
        self.all_means = tf.reshape(tf.transpose(means), [1, 49, 160])
        self.all_components = tf.transpose(components, perm=[0, 2, 1])

    @tf.function
    def call(self, X):
        return tf.transpose(tf.map_fn(
            lambda inp: inp[0] @ inp[1],
            (tf.transpose(X - self.all_means, perm=[2, 0, 1]), self.all_components),
            fn_output_signature=tf.TensorSpec(shape=[None, 5])
        ), perm=[1, 2, 0])

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape((batch_input_shape[0], 5, 160))

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "mean": self.means.numpy(), "components": self.components.numpy()}


all_means = np.array([filter.mean_ for filter in pca_per_filter])
all_components = np.array([filter.components_ for filter in pca_per_filter])

model_pcadense = keras.models.Sequential([
    keras.layers.Input(shape=(224, 224, 3), dtype=tf.uint8),
    keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float32)),
    keras.layers.Lambda(
        tf.keras.applications.mobilenet_v2.preprocess_input,
        name='preprocessing',
        input_shape=(224, 224, 3),
        dtype=tf.float32),
    feature_extractor,
    keras.layers.Reshape(target_shape=[49, 160]),
    PCATransformer(tf.constant(all_means), tf.constant(all_components)),
    keras.layers.Flatten(input_shape=[5, 160]),
    model_pca_based
])

model_pcasepconv = keras.models.Sequential([
    keras.layers.Input(shape=(224, 224, 3), dtype=tf.uint8),
    keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float32)),
    keras.layers.Lambda(
        tf.keras.applications.mobilenet_v2.preprocess_input,
        name='preprocessing',
        input_shape=(224, 224, 3),
        dtype=tf.float32),
    feature_extractor,
    keras.layers.Reshape(target_shape=[49, 160]),
    PCATransformer(tf.constant(all_means), tf.constant(all_components)),
    model_pca_sepconv
])

In [ ]:
pca_transformation = PCATransformer(tf.constant(all_means), tf.constant(all_components))

In [ ]:
lda_per_filter = [
    LDA(n_components=5).fit(X_train[..., i], np.argmax(y_train, axis=1))
    for i in range(X_train.shape[-1])
]

class LDATransformer(keras.layers.Layer):
    def __init__(self, means, components, **kwargs):
        super().__init__(**kwargs)
        self.all_means = tf.reshape(tf.transpose(means), [1, 49, 160])
        self.all_components = tf.transpose(components, perm=[0, 1, 2])

    @tf.function
    def call(self, X):
        return tf.transpose(tf.map_fn(
            lambda inp: inp[0] @ inp[1],
            (tf.transpose(X - self.all_means, perm=[2, 0, 1]), self.all_components),
            fn_output_signature=tf.TensorSpec(shape=[None, 5])
        ), perm=[1, 2, 0])

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape((batch_input_shape[0], 5, 160))

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "mean": self.means.numpy(), "components": self.components.numpy()}


all_means = np.array([filter.xbar_ for filter in lda_per_filter], dtype='float32')
all_components = np.array([filter.scalings_[:, :5] for filter in lda_per_filter], dtype='float32')

In [ ]:
lda_transformation = LDATransformer(tf.constant(all_means), tf.constant(all_components))

In [ ]:
def top_k_accuracy_score(y_true, y_pred, k=1):
    ranks = np.sum(y_pred >= y_pred[y_true == 1].reshape(-1, 1), axis=1)
    return np.sum(ranks <= k) / ranks.shape[0]

In [ ]:
print("Validation set accuracy")
preds_valid_1 = model_mobilenetv2_regularized(tf.reshape(X_valid, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_1, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_1, k=5))

print("Test set")
preds_test_aug_1 = model_mobilenetv2_regularized(tf.reshape(X_test, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_1, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_1, k=5))

Validation set accuracy
TOP 1 0.9747899159663865
TOP 5 0.9957983193277311
Test set
TOP 1 0.9572127139364304
TOP 5 0.9902200488997555


In [ ]:
print("Validation set accuracy")
preds_valid_2 = model_squeeze_excite_resnet(tf.reshape(X_valid, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_2, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_2, k=5))

print("Test set")
preds_test_aug_2 = model_squeeze_excite_resnet(tf.reshape(X_test, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_2, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_2, k=5))

Validation set accuracy
TOP 1 0.9229691876750701
TOP 5 0.9943977591036415
Test set
TOP 1 0.91320293398533
TOP 5 0.988997555012225


In [ ]:
print("Validation set accuracy")
preds_valid_3 = model_pca_based(tf.reshape(pca_transformation(X_valid), [-1, 800])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_3, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_3, k=5))

print("Test set")
preds_test_aug_3 = model_pca_based(tf.reshape(pca_transformation(X_test), [-1, 800])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_3, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_3, k=5))

Validation set accuracy
TOP 1 0.9117647058823529
TOP 5 0.9845938375350141
Test set
TOP 1 0.9058679706601467
TOP 5 0.9779951100244498


In [ ]:
print("Validation set accuracy")
preds_valid_4 = model_pca_sepconv(tf.reshape(pca_transformation(X_valid), [-1, 5, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_4, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_4, k=5))

print("Test set")
preds_test_aug_4 = model_pca_sepconv(tf.reshape(pca_transformation(X_test), [-1, 5, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_4, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_4, k=5))

Validation set accuracy
TOP 1 0.9243697478991597
TOP 5 0.9929971988795518
Test set
TOP 1 0.9144254278728606
TOP 5 0.9865525672371638


In [ ]:
print("Validation set accuracy")
preds_valid_4 = model_lda_based(tf.reshape(lda_transformation(X_valid), [-1, 800])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_4, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_4, k=5))

print("Test set")
preds_test_aug_4 = model_lda_based(tf.reshape(lda_transformation(X_test), [-1, 800])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_4, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_4, k=5))

Validation set accuracy
TOP 1 0.9047619047619048
TOP 5 0.9873949579831933
Test set
TOP 1 0.8948655256723717
TOP 5 0.9792176039119804


In [ ]:
print("Validation set accuracy")
preds_valid_5 = model_lda_sepconv(tf.reshape(lda_transformation(X_valid), [-1, 5, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_5, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_5, k=5))

print("Test set")
preds_test_aug_4 = model_lda_sepconv(tf.reshape(lda_transformation(X_test), [-1, 5, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_4, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_4, k=5))

Validation set accuracy
TOP 1 0.9285714285714286
TOP 5 0.9901960784313726
Test set
TOP 1 0.8973105134474327
TOP 5 0.9828850855745721
